In [1]:
using DifferentialEquations
using ReactionMechanismSimulator
using PyPlot

In [ ]:
outdict = readinput("chem43.rms")


In [3]:
liqspcs = outdict["gas"]["Species"]
liqrxns = outdict["gas"]["Reactions"]
surfspcs = outdict["surface"]["Species"]
surfrxns = outdict["surface"]["Reactions"]
interfacerxns = outdict[Set(["surface", "gas"])]["Reactions"]
solv = outdict["Solvents"][1];

In [4]:
liq = IdealDiluteSolution(liqspcs,liqrxns,solv,name="liquid",diffusionlimited=true);
surf = IdealSurface(surfspcs,surfrxns,3.121e-05,name="surface");

In [19]:
initialcondsliq = Dict(["proton"=>10.0^-4,
        "V"=>1.0,"T"=>298.15,"Phi"=>0.0,"d"=>0.0]);
AVratio = 1.0e5
initialcondssurf = Dict(["CO2X"=>0.1*3.121e-05*AVratio,
        "CHO2X"=>0.2*3.121e-05*AVratio,
        "CO2HX"=>0.2*3.121e-05*AVratio,
        "OX"=>0.1*3.121e-05*AVratio,
        "OCX"=>0.1*3.121e-05*AVratio,
        "vacantX"=>0.2*3.121e-05*AVratio,
        "CH2O2X"=>0.05*3.121e-05*AVratio,
        "CHO2X"=>0.04*3.121e-05*AVratio,
        "CH2OX"=>0.01*3.121e-05*AVratio,
        "A"=>1.0*AVratio,"T"=>298.15,"Phi"=>1.2]);

In [20]:
domainliq,y0liq,pliq = ConstantTVDomain(phase=liq,
    initialconds=initialcondsliq,constantspecies=["proton"]);
domaincat,y0cat,pcat = ConstantTAPhiDomain(phase=surf,
    initialconds=initialcondssurf);

In [21]:
inter,pinter = ReactiveInternalInterfaceConstantTPhi(domainliq,
  domaincat,interfacerxns,298.15,AVratio*1.0);

In [22]:
react,y0,p = Reactor((domainliq,domaincat), (y0liq,y0cat), (0.0, 1.0), [inter], (pliq,pcat,pinter));


In [ ]:
@time sol = solve(react.ode,DifferentialEquations.CVODE_BDF(),abstol=1e-16,reltol=1e-6);

In [ ]:
sol.retcode

In [11]:
ssys = SystemSimulation(sol,(domainliq,domaincat,),(inter,),p);

In [ ]:
rops(ssys,"CH2O2X",1)

In [ ]:
ssys.reactions[16]

In [ ]:
#Plot surface fractions
plotmolefractions(ssys.sims[2];tol=1e-9)
xlim(0.0,1)
gcf()

In [ ]:
Dict([ssys.sims[2].names[i]=>molefractions(ssys.sims[2],1.0)[i] for i in 1:length(ssys.sims[2].names)])

In [ ]:
getfluxdiagram(ssys,1;speciesratetolerance=0.4)

In [ ]:
println(ssys.names)

In [ ]:
plotrops(ssys,"CH2O2X",1;N=15,tol=0.0)

In [ ]:
plotrops(ssys,"CHO2X",1;N=10,tol=0.0)

In [ ]:
plotrops(ssys,"CO2HX",1;N=10,tol=0.0)

In [ ]:
plotrops(ssys,"OX",1;N=10,tol=0.0)
gcf()

In [ ]:
plotrops(ssys,"OCX",1.0e-6)
gcf()

In [ ]:
for (i,rxn) in enumerate(inter.reactions)
    str = getrxnstr(rxn)
    kf = inter.kfs[i]
    krev = inter.krevs[i]
    Kc = kf/krev
    println(str)
    println("kf = $kf")
    println("krev = $krev")
    println("Kc = $Kc")
end

In [ ]:
for (i,rxn) in enumerate(inter.reactions)
    str = getrxnstr(rxn)
    kf = inter.kfs[i]
    krev = inter.krevs[i]
    Kc = kf/krev
    println(str)
    println("kf = $kf")
    println("krev = $krev")
    println("Kc = $Kc")
end